# Feature Engineering & Data Cleaning

**Goal**: Prepare the LendingClub dataset for Deep Learning and Offline Reinforcement Learning models.

**Key Steps**:
1.  **Target Definition**: 'Fully Paid' (0) vs. 'Charged Off'/'Default' (1).
2.  **Leakage Removal**: Remove future-looking variables (e.g., specific payments, recoveries).
3.  **Feature Engineering**: Create ratios, handle dates, process categorical risk factors.
4.  **Cleaning & Preprocessing**: Handle missing values, encode categoricals, and scale features.
5.  **Output**: Save processed train/test sets.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Configuration
DATA_PATH = Path("data/accepted_2007_to_2018Q4.csv")
PROCESSED_DATA_DIR = Path("data/processed")
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Set to 200000 (approx 10% of data) to fit comfortably within 16GB RAM.
# Set to None ONLY if you have >32GB RAM available.
SAMPLE_SIZE = False

pd.set_option("display.max_columns", None)

## 1. Load Data

In [12]:
print("Loading dataset...")
if SAMPLE_SIZE:
    # Load a chunk for efficiency
    df_iter = pd.read_csv(DATA_PATH, low_memory=False, chunksize=SAMPLE_SIZE)
    df = next(df_iter)
    print(f"Loaded SAMPLE of {SAMPLE_SIZE} rows.")
else:
    df = pd.read_csv(DATA_PATH, low_memory=False)
    print(f"Loaded FULL dataset.")

print(f"Initial shape: {df.shape}")

Loading dataset...
Loaded FULL dataset.
Initial shape: (2260701, 151)


## 2. Target Definition
We only consider completed loans: 'Fully Paid' and 'Charged Off' (including 'Default').
We drop 'Current' loans as their final status is unknown.

In [13]:
target_status = ['Fully Paid', 'Charged Off', 'Default']
df = df[df['loan_status'].isin(target_status)].copy()

# Map to Binary Target: 0 = Fully Paid, 1 = Default/Charged Off
df['target'] = df['loan_status'].apply(lambda x: 0 if x == 'Fully Paid' else 1)

print(f"Shape after filtering target: {df.shape}")
print("Target Distribution:")
print(df['target'].value_counts(normalize=True))

Shape after filtering target: (1345350, 152)
Target Distribution:
target
0    0.80035
1    0.19965
Name: proportion, dtype: float64


## 3. Data Cleaning: Remove Leakage & Irrelevant Features
We diligently remove features that are not available at the time of application ($t=0$). 

In [14]:
leakage_cols = [
    # Funding info (post-approval)
    'funded_amnt', 'funded_amnt_inv',
    
    # Payment tracking (future info)
    'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
    'recoveries', 'collection_recovery_fee',
    'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
    'last_credit_pull_d',
    
    # Current Status (implies non-default)
    'out_prncp', 'out_prncp_inv',
    
    # Settlement & Hardship (only for distressed loans)
    'debt_settlement_flag', 'debt_settlement_flag_date', 
    'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term',
    'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_length', 
    'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 
    'hardship_dpd', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'orig_projected_additional_accrued_interest',
    'chargeoff_within_12_mths',
    'collections_12_mths_ex_med'
]

irrelevant_cols = [
    'id', 'member_id', 'url', 'desc',
    'title', 'zip_code', 'policy_code',
    'emp_title' # High cardinality feature that causes OHE explosion
]

cols_to_drop = leakage_cols + irrelevant_cols
# Only drop columns that actually exist in the dataframe
cols_to_drop = [c for c in cols_to_drop if c in df.columns]

df.drop(columns=cols_to_drop, inplace=True)
print(f"Dropped {len(cols_to_drop)} columns. New shape: {df.shape}")

Dropped 45 columns. New shape: (1345350, 107)


## 4. Feature Engineering
Creating new features and transforming existing ones for better model performance.

In [15]:
# 1. Term to Numeric
df['term'] = df['term'].str.replace(' months', '').astype(float)

# 2. Emp Length to Numeric
emp_map = {
    '< 1 year': 0,
    '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4, '5 years': 5,
    '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10
}
df['emp_length'] = df['emp_length'].map(emp_map)

# 3. Date Features
# Ensure datetime
df['issue_d'] = pd.to_datetime(df['issue_d'])
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])

# Credit History Length (in years)
df['credit_history_years'] = (df['issue_d'] - df['earliest_cr_line']).dt.days / 365.25
# Handle potential negative values or errors if issue_d < earliest_cr_line (rare data errors)
df['credit_history_years'] = df['credit_history_years'].clip(lower=0)

# 4. FICO Score Average
df['fico_score'] = 0.5 * (df['fico_range_low'] + df['fico_range_high'])
df.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)

# 5. New Risk Indicators
df['loan_to_income'] = df['loan_amnt'] / df['annual_inc']
df['revol_bal_to_income'] = df['revol_bal'] / df['annual_inc']
df['has_delinquency'] = (df['delinq_2yrs'] > 0).astype(int)

# Handle inf values created by division by zero if annual_inc is 0
df.replace([np.inf, -np.inf], np.nan, inplace=True)

print("Feature engineering complete.")

C:\Users\harsh\AppData\Local\Temp\ipykernel_17356\920015685.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['issue_d'] = pd.to_datetime(df['issue_d'])
C:\Users\harsh\AppData\Local\Temp\ipykernel_17356\920015685.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])


Feature engineering complete.


## 5. Handling Missing Values
Dropping high-missingness columns (except critical ones) and imputing the rest.

In [16]:
# Calculate missing %
missing_pct = df.isnull().mean() * 100

# Drop columns with > 50% missing, conserving important ones if needed
# (User feedback: 'emp_length' is predictive, we kept it and mapped it, now we will impute it)
drop_threshold = 50
cols_to_drop_missing = missing_pct[missing_pct > drop_threshold].index.tolist()

# Explicitly KEEP emp_length even if it was > 50% (it's around 6% usually, but good to be safe)
if 'emp_length' in cols_to_drop_missing:
    cols_to_drop_missing.remove('emp_length')

df.drop(columns=cols_to_drop_missing, inplace=True)
print(f"Dropped {len(cols_to_drop_missing)} columns due to missingness > {drop_threshold}%.")

Dropped 37 columns due to missingness > 50%.


## 6. Train/Test Split (Temporal awareness)
We use `issue_d` to split, then drop it from the feature set.

In [17]:
# Sort by date to simulate real-world 'past predicting future' scenario if strictly needed,
# or just random split if we assume stationarity. 
# Included `issue_d` preservation plan: Keep relevant for analysis.
# For this baseline, we'll do a standard random split but keep date for analysis if needed.
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# Drop issue_d and earliest_cr_line from inputs
date_cols = ['issue_d', 'earliest_cr_line']
train_df = train_df.drop(columns=date_cols)
test_df = test_df.drop(columns=date_cols)

Train shape: (1076280, 73)
Test shape: (269070, 73)


## 7. Preprocessing Pipeline
- **Imputation**: Median for numeric, 'Unknown' for categorical.
- **Encoding**: Ordinal for 'grade', One-Hot for others.
- **Scaling**: Standard Scaler.

In [18]:
# Define Features
y_train = train_df['target']
y_test = test_df['target']
X_train = train_df.drop(columns=['target', 'loan_status'])
X_test = test_df.drop(columns=['target', 'loan_status'])

# Identify column types
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Specific handling for Grade (Ordinal)
if 'grade' in categorical_features:
    categorical_features.remove('grade')
    ordinal_features = ['grade']
else:
    ordinal_features = []

print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")
print(f"Ordinal features: {len(ordinal_features)}")

Numeric features: 58
Categorical features: 9
Ordinal features: 1


In [19]:
# Define Transformers

# Numeric: Impute Median -> Scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical High Card/Nominal: Impute 'Unknown' -> OneHot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Ordinal (Grade): Impute -> Ordinal Encode
grade_order = [['A', 'B', 'C', 'D', 'E', 'F', 'G']]
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=grade_order))
])

# Combine
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ],
    verbose_feature_names_out=False
)

# Fit & Transform
# Note: We fit on Train, transform both
print("Fitting preprocessor...")
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Get feature names back
try:
    feature_names = preprocessor.get_feature_names_out()
except AttributeError:
    # Fallback for older sklearn versions if needed, though usually not an issue with updated envs
    feature_names = numeric_features + list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)) + ordinal_features

# Convert back to DataFrame
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print(f"Processed Train shape: {X_train_df.shape}")

Fitting preprocessor...
Processed Train shape: (1076280, 175)


## 8. Save Processed Data

In [20]:
print("Saving processed data...")
# Save compressed to save space
X_train_df.to_parquet(PROCESSED_DATA_DIR / "X_train.parquet")
X_test_df.to_parquet(PROCESSED_DATA_DIR / "X_test.parquet")
y_train.to_frame().to_parquet(PROCESSED_DATA_DIR / "y_train.parquet")
y_test.to_frame().to_parquet(PROCESSED_DATA_DIR / "y_test.parquet")

print("Data saved to data/processed/")

Saving processed data...
Data saved to data/processed/
